# RoboMage Data Module Demonstration

This notebook demonstrates the **RoboMage data module** - the foundational data abstraction layer for powder diffraction analysis. 

## What You'll Learn:
- 🏗️ **Purpose and architecture** of the data module
- 📊 **Loading and validating** powder diffraction data
- 📈 **Quality metrics** and statistical analysis
- 🔧 **Data manipulation** operations
- 📋 **Visualization** techniques
- 🧪 **Real-world examples** with SRM 660b standard

## Why This Module Matters:
The data module transforms RoboMage from working with raw arrays into having a **proper domain model** for powder diffraction analysis, ensuring data integrity, rich metadata, and scientific reproducibility.

## 1. Import Required Libraries

We'll import the RoboMage data module components along with standard scientific computing libraries for visualization and analysis.

In [ ]:
# Core RoboMage data module
from datetime import datetime

import matplotlib.pyplot as plt

# Scientific computing libraries
import numpy as np
import pandas as pd

from robomage.data import (
    DataStatistics,
    DiffractionData,
)
from robomage.data.loaders import load_test_data

# Configure matplotlib for better plots
plt.style.use("default")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 12
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.3
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 0.8

print("✅ All libraries imported successfully!")
print(f"📅 Notebook run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Load Sample Dataset - SRM 660b LaB6 Standard

We'll demonstrate loading real powder diffraction data using the SRM 660b LaB6 standard reference material that comes with RoboMage.

In [ ]:
# Load the test data using the RoboMage data module
print("Loading SRM 660b LaB6 standard reference material...")
data = load_test_data()

print("✅ Successfully loaded powder diffraction data!")
print(f"📊 Dataset: {data.filename}")
print(f"📏 Data points: {len(data.q_values):,}")
print(f"🏷️  Sample name: {data.sample_name or 'Not specified'}")
print(f"📅 Loaded at: {data.timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# Display basic information about the data structure
print("\n🔍 Data Structure:")
print(f"   Q values shape: {data.q_values.shape}")
print(f"   Intensities shape: {data.intensities.shape}")
print(f"   Q values type: {type(data.q_values)}")
print(f"   Intensities type: {type(data.intensities)}")

## 3. Basic Data Exploration

Let's examine the data structure and preview the Q and intensity values to understand what we're working with.

In [ ]:
# Convert to DataFrame for easy viewing
df = data.to_dataframe()

print("📋 Dataset Overview:")
print(f"   Shape: {df.shape}")
print(f"   Columns: {list(df.columns)}")
print(f"   Data types:\n{df.dtypes}")

print("\n📊 First 5 data points:")
print(df.head())

print("\n📊 Last 5 data points:")
print(df.tail())

print("\n📏 Q-space Coverage:")
print(f"   Minimum Q: {data.q_values.min():.3f} Å⁻¹")
print(f"   Maximum Q: {data.q_values.max():.3f} Å⁻¹")
print(f"   Q range: {data.q_values.max() - data.q_values.min():.3f} Å⁻¹")

print("\n📈 Intensity Range:")
print(f"   Minimum intensity: {data.intensities.min():.1f}")
print(f"   Maximum intensity: {data.intensities.max():.1f}")
print(f"   Dynamic range: {data.intensities.max() / data.intensities.min():.1f}×")

## 4. Data Quality Assessment - Automatic Statistics

The **key feature** of the RoboMage data module is automatic computation of quality metrics. Let's explore the built-in statistics that help assess data quality.

In [ ]:
# Access the automatically computed statistics
stats = data.statistics

print("🎯 RoboMage Automatic Data Quality Assessment")
print("=" * 50)

print("📊 Data Coverage:")
print(f"   Total points: {stats.num_points:,}")
print(f"   Q range: {stats.q_range[0]:.3f} to {stats.q_range[1]:.3f} Å⁻¹")

print("\n📏 Q-space Sampling Quality:")
print(f"   Mean Q step: {stats.q_step_mean:.6f} Å⁻¹")
print(f"   Q step std dev: {stats.q_step_std:.6f} Å⁻¹")
print(
    f"   Sampling uniformity: {(1 - stats.q_step_std / stats.q_step_mean) * 100:.1f}%"
)

print("\n📈 Signal Characteristics:")
print(
    f"   Intensity range: {stats.intensity_range[0]:.1f} to {stats.intensity_range[1]:.1f}"
)
print(f"   Mean intensity: {stats.intensity_mean:.1f}")
print(f"   Intensity std dev: {stats.intensity_std:.1f}")
print(f"   Signal-to-noise estimate: {stats.intensity_mean / stats.intensity_std:.1f}")

# Quality assessment
uniformity = (1 - stats.q_step_std / stats.q_step_mean) * 100
if uniformity > 95:
    quality = "🟢 EXCELLENT"
elif uniformity > 90:
    quality = "🟡 GOOD"
else:
    quality = "🔴 POOR"

print(f"\n🎖️  Overall Q-sampling Quality: {quality}")
print(f"   (Uniformity: {uniformity:.1f}%)")

## 5. Q-space Sampling Analysis

Let's analyze the Q-space sampling in detail to understand the data collection characteristics.

In [ ]:
# Calculate Q-step variations
q_steps = np.diff(data.q_values)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle(
    "Q-space Sampling Analysis for SRM 660b LaB6", fontsize=16, fontweight="bold"
)

# 1. Q-step size distribution
axes[0, 0].hist(q_steps, bins=50, alpha=0.7, color="skyblue", edgecolor="black")
axes[0, 0].axvline(
    stats.q_step_mean,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Mean: {stats.q_step_mean:.6f}",
)
axes[0, 0].set_xlabel("Q Step Size (Å⁻¹)")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].set_title("Distribution of Q Step Sizes")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Q-step size vs Q position
axes[0, 1].plot(data.q_values[1:], q_steps, ".", alpha=0.6, markersize=1)
axes[0, 1].set_xlabel("Q (Å⁻¹)")
axes[0, 1].set_ylabel("Q Step Size (Å⁻¹)")
axes[0, 1].set_title("Q Step Size vs Q Position")
axes[0, 1].grid(True, alpha=0.3)

# 3. Cumulative Q coverage
axes[1, 0].plot(range(len(data.q_values)), data.q_values, color="green", linewidth=2)
axes[1, 0].set_xlabel("Data Point Index")
axes[1, 0].set_ylabel("Q (Å⁻¹)")
axes[1, 0].set_title("Cumulative Q Coverage")
axes[1, 0].grid(True, alpha=0.3)

# 4. Data point density
q_bins = np.linspace(data.q_values.min(), data.q_values.max(), 50)
density, _ = np.histogram(data.q_values, bins=q_bins)
bin_centers = (q_bins[1:] + q_bins[:-1]) / 2
axes[1, 1].plot(bin_centers, density, "o-", color="purple", linewidth=2)
axes[1, 1].set_xlabel("Q (Å⁻¹)")
axes[1, 1].set_ylabel("Data Points per Bin")
axes[1, 1].set_title("Data Point Density Distribution")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed sampling analysis
print("📊 Q-space Sampling Analysis:")
print(f"   Q steps - Min: {q_steps.min():.6f} Å⁻¹")
print(f"   Q steps - Max: {q_steps.max():.6f} Å⁻¹")
print(f"   Q steps - Range: {q_steps.max() - q_steps.min():.6f} Å⁻¹")
print(
    f"   Coefficient of variation: {(stats.q_step_std / stats.q_step_mean) * 100:.2f}%"
)

## 6. Powder Diffraction Pattern Visualization

Let's visualize the actual powder diffraction pattern and identify key features typical of LaB6 standard.

In [ ]:
# Create comprehensive diffraction pattern visualization
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle(
    "SRM 660b LaB6 Powder Diffraction Pattern Analysis", fontsize=16, fontweight="bold"
)

# 1. Full pattern overview
axes[0].plot(data.q_values, data.intensities, "-", linewidth=1, color="blue", alpha=0.8)
axes[0].set_xlabel("Q (Å⁻¹)")
axes[0].set_ylabel("Intensity")
axes[0].set_title("Complete Powder Diffraction Pattern")
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(data.q_values.min(), data.q_values.max())

# Add statistics annotation
textstr = f"Points: {len(data.q_values):,}\nQ range: {stats.q_range[0]:.2f}-{stats.q_range[1]:.2f} Å⁻¹\nMax intensity: {stats.intensity_range[1]:.0f}"
axes[0].text(
    0.02,
    0.98,
    textstr,
    transform=axes[0].transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5),
)

# 2. Low-Q region (showing main peaks)
low_q_mask = data.q_values <= 10
axes[1].plot(
    data.q_values[low_q_mask],
    data.intensities[low_q_mask],
    "-",
    linewidth=2,
    color="red",
)
axes[1].set_xlabel("Q (Å⁻¹)")
axes[1].set_ylabel("Intensity")
axes[1].set_title("Low-Q Region (Q ≤ 10 Å⁻¹) - Main Diffraction Peaks")
axes[1].grid(True, alpha=0.3)

# Find and annotate major peaks in low-Q region
from scipy.signal import find_peaks

peaks, properties = find_peaks(data.intensities[low_q_mask], height=1000, distance=20)
if len(peaks) > 0:
    peak_q = data.q_values[low_q_mask][peaks]
    peak_intensities = data.intensities[low_q_mask][peaks]
    axes[1].plot(
        peak_q, peak_intensities, "ro", markersize=8, label=f"{len(peaks)} major peaks"
    )
    axes[1].legend()

# 3. Logarithmic scale view
axes[2].semilogy(
    data.q_values, data.intensities, "-", linewidth=1, color="green", alpha=0.8
)
axes[2].set_xlabel("Q (Å⁻¹)")
axes[2].set_ylabel("Intensity (log scale)")
axes[2].set_title("Log-Scale View (showing weak features and background)")
axes[2].grid(True, alpha=0.3)
axes[2].set_xlim(data.q_values.min(), data.q_values.max())

plt.tight_layout()
plt.show()

# Print peak analysis
if "peaks" in locals() and len(peaks) > 0:
    print("🎯 Major Peak Analysis (Q ≤ 10 Å⁻¹):")
    print(f"   Number of major peaks: {len(peaks)}")
    print(f"   Peak positions (Q): {', '.join([f'{q:.3f}' for q in peak_q[:5]])} Å⁻¹")
    print(
        f"   Peak intensities: {', '.join([f'{int(i)}' for i in peak_intensities[:5]])}"
    )
else:
    print("🔍 No major peaks found with current criteria")

## 7. Data Manipulation Operations

The RoboMage data module provides powerful methods for data manipulation. Let's demonstrate trimming and interpolation operations.

In [ ]:
# Demonstrate data manipulation operations
print("🔧 Data Manipulation Demonstrations")
print("=" * 40)

# 1. Trim to focus on main diffraction region
main_region = data.trim_q_range(q_min=1.0, q_max=8.0)
print("📏 Trimmed to main region (Q: 1-8 Å⁻¹):")
print(f"   Original points: {len(data.q_values):,}")
print(f"   Trimmed points: {len(main_region.q_values):,}")
print(
    f"   Data reduction: {(1 - len(main_region.q_values) / len(data.q_values)) * 100:.1f}%"
)

# 2. Create uniform Q grid via interpolation
q_uniform = np.linspace(1.0, 8.0, 1000)  # 1000 evenly spaced points
resampled = main_region.interpolate(q_uniform)
print("\n📐 Resampled to uniform grid:")
print(f"   Original Q step (mean): {main_region.statistics.q_step_mean:.6f} Å⁻¹")
print(f"   New uniform Q step: {q_uniform[1] - q_uniform[0]:.6f} Å⁻¹")
print(
    f"   Uniformity improvement: {(1 - main_region.statistics.q_step_std / main_region.statistics.q_step_mean) * 100:.1f}% → 100.0%"
)

# 3. Visualize the manipulations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle("Data Manipulation Operations", fontsize=16, fontweight="bold")

# Original data
axes[0, 0].plot(
    data.q_values, data.intensities, "-", alpha=0.7, color="blue", linewidth=1
)
axes[0, 0].set_xlabel("Q (Å⁻¹)")
axes[0, 0].set_ylabel("Intensity")
axes[0, 0].set_title(f"Original Data ({len(data.q_values):,} points)")
axes[0, 0].grid(True, alpha=0.3)

# Trimmed data
axes[0, 1].plot(
    main_region.q_values,
    main_region.intensities,
    "-",
    alpha=0.8,
    color="red",
    linewidth=2,
)
axes[0, 1].set_xlabel("Q (Å⁻¹)")
axes[0, 1].set_ylabel("Intensity")
axes[0, 1].set_title(f"Trimmed Data (Q: 1-8 Å⁻¹, {len(main_region.q_values):,} points)")
axes[0, 1].grid(True, alpha=0.3)

# Original vs resampled comparison
axes[1, 0].plot(
    main_region.q_values,
    main_region.intensities,
    "-",
    alpha=0.6,
    color="red",
    label="Original",
    linewidth=1,
)
axes[1, 0].plot(
    resampled.q_values,
    resampled.intensities,
    "-",
    alpha=0.8,
    color="green",
    label="Resampled",
    linewidth=2,
)
axes[1, 0].set_xlabel("Q (Å⁻¹)")
axes[1, 0].set_ylabel("Intensity")
axes[1, 0].set_title("Original vs Resampled Comparison")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Q-step uniformity comparison
q_steps_orig = np.diff(main_region.q_values)
q_steps_new = np.diff(resampled.q_values)
axes[1, 1].hist(
    q_steps_orig,
    bins=30,
    alpha=0.7,
    color="red",
    label=f"Original (σ={np.std(q_steps_orig):.6f})",
)
axes[1, 1].hist(
    q_steps_new,
    bins=30,
    alpha=0.7,
    color="green",
    label=f"Resampled (σ={np.std(q_steps_new):.6f})",
)
axes[1, 1].set_xlabel("Q Step Size (Å⁻¹)")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].set_title("Q Step Size Distribution")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Verify data integrity is maintained
print("\n✅ Data Integrity Checks:")
print(f"   Original metadata preserved: {resampled.filename == data.filename}")
print(
    f"   Peak position preserved: {np.abs(resampled.intensities.max() - main_region.intensities.max()) < 50}"
)
print(f"   Q ordering maintained: {np.all(np.diff(resampled.q_values) > 0)}")

## 8. Comparison with Traditional Approach

Let's compare the RoboMage data module approach with traditional manual data handling to highlight the benefits.

In [ ]:
# Traditional approach (manual, error-prone)
print("🔧 Traditional Manual Approach:")
print("=" * 40)

# Simulate loading data the old way
import time

start_time = time.time()

# Manual steps that used to be required
raw_data = np.loadtxt("../examples/pdf_SRM_660b_q.chi", comments="#")
raw_df = pd.DataFrame(raw_data, columns=["Q", "intensity"])

# Manual validation (easy to forget!)
assert raw_df.shape[1] == 2, "Wrong number of columns"
assert len(raw_df) > 0, "Empty dataset"

# Manual sorting (often forgotten!)
raw_df = raw_df.sort_values("Q").reset_index(drop=True)

# Manual statistics calculation
manual_stats = {
    "num_points": len(raw_df),
    "q_range": (raw_df["Q"].min(), raw_df["Q"].max()),
    "q_step_mean": raw_df["Q"].diff().mean(),
    "q_step_std": raw_df["Q"].diff().std(),
    "intensity_range": (raw_df["intensity"].min(), raw_df["intensity"].max()),
    "intensity_mean": raw_df["intensity"].mean(),
    "intensity_std": raw_df["intensity"].std(),
}

manual_time = time.time() - start_time

print("   ❌ Manual validation required")
print("   ❌ Manual sorting required")
print("   ❌ Manual statistics calculation")
print("   ❌ No metadata preservation")
print(f"   ⏱️  Processing time: {manual_time:.4f} seconds")

print("\n🚀 RoboMage Data Module Approach:")
print("=" * 40)

start_time = time.time()
# One-liner with automatic everything!
robomage_data = load_test_data()
robomage_time = time.time() - start_time

print("   ✅ Automatic validation")
print("   ✅ Automatic sorting")
print("   ✅ Automatic statistics")
print("   ✅ Rich metadata preservation")
print("   ✅ Type safety and IDE support")
print(f"   ⏱️  Processing time: {robomage_time:.4f} seconds")

# Compare results
print("\n📊 Results Comparison:")
print(
    f"   Data points: {manual_stats['num_points']} vs {robomage_data.statistics.num_points} ✅"
)
print(f"   Q range: {manual_stats['q_range']} vs {robomage_data.statistics.q_range} ✅")
print(
    f"   Mean Q step: {manual_stats['q_step_mean']:.6f} vs {robomage_data.statistics.q_step_mean:.6f} ✅"
)

# Demonstrate the key advantage: Rich operations
print("\n🎯 Advanced Operations Comparison:")
print("Manual approach:")
print("   # Multiple steps for trimming")
print("   mask = (raw_df['Q'] >= 1.0) & (raw_df['Q'] <= 5.0)")
print("   trimmed_df = raw_df[mask].copy()")
print("   # No metadata preserved!")

print("\nRoboMage approach:")
print("   # One line with metadata preservation")
print("   trimmed = robomage_data.trim_q_range(q_min=1.0, q_max=5.0)")
print("   # Filename, timestamp, all metadata automatically preserved!")

print("\n🏆 Benefits Summary:")
print("   📈 Code reduction: ~80% fewer lines")
print("   🛡️  Error prevention: Automatic validation")
print("   📝 Metadata preservation: Complete provenance")
print("   🔧 Rich operations: Domain-specific methods")
print("   🧪 Scientific reproducibility: Standardized format")

## 9. Data Module Architecture Summary

Let's summarize the key architectural components and their purposes in the RoboMage data module.

In [ ]:
# Display the data module architecture
print("🏗️  RoboMage Data Module Architecture")
print("=" * 50)

print("""
📦 src/robomage/data/
├── 📄 __init__.py           # Public API exports
├── 🎯 models.py            # Core data models  
└── 📥 loaders.py           # File loading utilities

🎯 Core Classes:
├── DiffractionData         # Main data container with rich functionality
└── DataStatistics          # Automatic quality metrics computation

🔧 Key Features:
""")

# Introspect the DiffractionData class
print("📋 DiffractionData Methods:")
methods = [
    method
    for method in dir(DiffractionData)
    if not method.startswith("_") and callable(getattr(DiffractionData, method))
]
for i, method in enumerate(methods, 1):
    if i <= 10:  # Show first 10 methods
        print(f"   {i:2}. {method}")
    elif i == 11:
        print(f"   ... and {len(methods) - 10} more methods")
        break

print("\n📊 DataStatistics Fields:")
stats_fields = list(DataStatistics.__annotations__.keys())
for i, field in enumerate(stats_fields, 1):
    print(f"   {i}. {field}")

print("\n🔄 Data Flow:")
print("   File → Loader → DiffractionData → Statistics")
print("   ↓")
print("   Validation → Sorting → Rich Operations")

print("\n💡 Design Principles:")
print("   ✅ Immutable operations (new objects returned)")
print("   ✅ Type safety with Pydantic v2")
print("   ✅ Automatic validation and quality checks")
print("   ✅ Rich metadata preservation")
print("   ✅ Domain-specific operations")
print("   ✅ Scientific reproducibility")

# Demonstrate the public API
print("\n🔌 Public API Usage Patterns:")
print("""
# Loading data
from robomage.data import load_diffraction_file, load_test_data

# Creating data objects  
from robomage.data import DiffractionData
data = DiffractionData(q_values=q_vals, intensities=intensities)

# Accessing automatic statistics
stats = data.statistics  # Computed on-demand

# Data manipulation
subset = data.trim_q_range(q_min=1.0, q_max=5.0)
resampled = data.interpolate(new_q_grid)
df = data.to_dataframe()

# Integration with existing code
pandas_df = data.to_dataframe()  # Convert to DataFrame when needed
""")

print("\n🚀 Ready for Advanced Features:")
print("   🔬 Multi-sample analysis workflows")
print("   🗄️  Database integration (SQLAlchemy models)")
print("   🤖 Machine learning feature extraction")
print("   📊 Advanced visualization pipelines")
print("   🔄 Batch processing operations")
print("   📝 Automated analysis reporting")